# 1. IO编程

IO在计算机中指Input/Output，也就是输入和输出。由于程序和运行时数据是在内存中驻留，由CPU这个超快的计算核心来执行，涉及到数据交换的地方，通常是磁盘、网络等，就需要IO接口。

IO编程中，Stream（流）是一个很重要的概念，可以把流想象成一个水管，数据就是水管里的水，但是只能单向流动。Input Stream就是数据从外面（磁盘、网络）流进内存，Output Stream就是数据从内存流到外面去。对于浏览网页来说，浏览器和Web服务器之间至少需要建立两根水管，才可以既能发数据，又能收数据。


由于CPU和内存的速度远远高于外设的速度，所以，在IO编程中，就存在速度严重不匹配的问题。举个例子来说，比如要把100M的数据写入磁盘，CPU输出100M的数据只需要0.01秒，可是磁盘要接收这100M数据可能需要10秒，怎么办呢？有两种办法：

第一种是CPU等着，也就是程序暂停执行后续代码，等100M的数据在10秒后写入磁盘，再接着往下执行，这种模式称为同步IO；

另一种方法是CPU不等待，只是告诉磁盘，“您老慢慢写，不着急，我接着干别的事去了”，于是，后续代码可以立刻接着执行，这种模式称为异步IO。

## 1.1 文件读写

### 1.1.1 写入测试文件：

In [1]:
%%writefile test.txt
this is a test file.
hello world!
python is good!
today is a good day.

Overwriting test.txt


### 1.1.2 读文件

读写文件前，我们先必须了解一下，在磁盘上读写文件的功能都是由操作系统提供的，现代操作系统不允许普通的程序直接操作磁盘，所以，读写文件就是请求操作系统打开一个文件对象（通常称为文件描述符），然后，通过操作系统提供的接口从这个文件对象中读取数据（读文件），或者把数据写入这个文件对象（写文件）

使用 `open` 函数来打开文件，使用文件名的字符串作为输入参数：

In [2]:
f = open('test.txt')  # f = open('test.txt'， ‘r’)

默认以读的方式打开文件，如果文件不存在会报错。

可以使用 `read` 方法来读入文件中的所有内容：

In [3]:
text = f.read()
print(text)

this is a test file.
hello world!
python is good!
today is a good day.


使用完文件之后，需要将文件关闭。

In [4]:
f.close()

也可以按照行读入内容，`readlines` 方法返回一个列表，每个元素代表文件中每一行的内容：

In [5]:
f = open('test.txt')
lines = f.readlines()
print(lines)
f.close()

['this is a test file.\n', 'hello world!\n', 'python is good!\n', 'today is a good day.']


事实上，我们可以将 `f` 放在一个循环中，得到它每一行的内容：

In [6]:
f = open('test.txt')
for line in f:
    print(line)
f.close()

this is a test file.

hello world!

python is good!

today is a good day.


删除刚才创建的文件：

In [7]:
import os
os.remove('test.txt')

### 1.1.3 写文件

我们使用 `open` 函数的写入模式来写文件：

In [ ]:
f = open('myfile.txt', 'w')
f.write('hello world!')
f.close()

使用 `w` 模式时，如果文件不存在会被创建，我们可以查看是否真的写入成功：

In [ ]:
print(open('myfile.txt').read())

如果文件已经存在， `w` 模式会覆盖之前写的所有内容：

In [ ]:
f = open('myfile.txt', 'w')
f.write('another hello world!')
f.close()
print(open('myfile.txt').read())

除了写入模式，还有追加模式 `a` ，追加模式不会覆盖之前已经写入的内容，而是在之后继续写入：

In [ ]:
f = open('myfile.txt', 'a')
f.write('... and more')
f.close()
print(open('myfile.txt').read())

写入结束之后一定要将文件关闭，否则可能出现内容没有完全写入文件中的情况。

还可以使用读写模式 `w+`：

In [ ]:
f = open('myfile.txt', 'w+')
f.write('hello world!')
f.seek(6)
print (f.read())
f.close()

这里 `f.seek(6)` 移动到文件的第6个字符处，然后 `f.read()` 读出剩下的内容。

In [ ]:
import os
os.remove('myfile.txt')

### 1.1.4  二进制文件

二进制读写模式 wb：

In [19]:
import os
f = open('binary.bin', 'wb')
f.write(os.urandom(4))  #随机生成n个字节的串
f.close()

f = open('binary.bin', 'rb')
result = repr(f.read())
print(result)   # repr()  返回一个对象的 string 格式 (字节字符串格式，而非文本字符串)
#print(f.read())
f.close()

b'\xfaA\xf5\x98'


In [ ]:
import os
os.remove('binary.bin')

如果想从二进制模式的文件中读取或写入文本数据，必须确保要进行解码和编码操作。

In [22]:
with open('somefile.bin', 'wb') as f:
    text = 'Hello World'
    f.write(text.encode('utf-8'))

with open('somefile.bin', 'rb') as f:
    data = f.read(16)
    text = data.decode('utf-8')
    print(text)

Hello World


### 1.1.5 字符编码

要读取非UTF-8编码的文本文件，需要给open()函数传入encoding参数，例如，读取GBK编码的文件：

In [ ]:
f = open('gbk.txt', 'r', encoding='gbk')


遇到有些编码不规范的文件，你可能会遇到UnicodeDecodeError，因为在文本文件中可能夹杂了一些非法编码的字符。遇到这种情况，open()函数还接收一个errors参数，表示如果遇到编码错误后如何处理。最简单的方式是直接忽略：

f = open('gbk.txt', 'r', encoding='gbk', errors='ignore')

### 1.1.6 关闭文件

在**Python**中，如果一个打开的文件不再被其他变量引用时，它会自动关闭这个文件。

所以正常情况下，如果一个文件正常被关闭了，忘记调用文件的 `close` 方法不会有什么问题。

关闭文件可以保证内容已经被写入文件，而不关闭可能会出现意想不到的结果：

In [23]:
f = open('newfile.txt','w')
f.write('hello world')
g = open('newfile.txt', 'r')
print(repr(g.read()))

''


虽然这里写了内容，但是在关闭之前，这个内容并没有被写入磁盘。

使用循环写入的内容也并不完整：

In [ ]:
f = open('newfile.txt','w')
for i in range(3000):
    f.write('hello world: ' + str(i) + '\n')

g = open('newfile.txt', 'r')
print (g.read())
f.close()
g.close()

In [25]:
import os
os.remove('newfile.txt')

出现异常时候的读写：

In [26]:
f = open('newfile.txt','w')
for i in range(3000):
    x = 1.0 / (i - 1000)
    f.write('hello world: ' + str(i) + '\n')

ZeroDivisionError: float division by zero

查看已有内容：

In [ ]:
g = open('newfile.txt', 'r')
print (g.read())
f.close()
g.close()

可以看到，出现异常的时候，磁盘的写入并没有完成，为此我们可以使用 `try/except/finally` 块来关闭文件，这里 `finally` 确保关闭文件，所有的写入已经完成。

In [31]:
f = open('newfile.txt','w')
try:
    for i in range(3000):
        x = 1.0 / (i - 1000)
        f.write('hello world: ' + str(i) + '\n')
except Exception:
    print("something bad happened")
finally:
    f.close()

something bad happened


In [32]:
g = open('newfile.txt', 'r')
print (g.read())
g.close()

hello world: 0
hello world: 1
hello world: 2
hello world: 3
hello world: 4
hello world: 5
hello world: 6
hello world: 7
hello world: 8
hello world: 9
hello world: 10
hello world: 11
hello world: 12
hello world: 13
hello world: 14
hello world: 15
hello world: 16
hello world: 17
hello world: 18
hello world: 19
hello world: 20
hello world: 21
hello world: 22
hello world: 23
hello world: 24
hello world: 25
hello world: 26
hello world: 27
hello world: 28
hello world: 29
hello world: 30
hello world: 31
hello world: 32
hello world: 33
hello world: 34
hello world: 35
hello world: 36
hello world: 37
hello world: 38
hello world: 39
hello world: 40
hello world: 41
hello world: 42
hello world: 43
hello world: 44
hello world: 45
hello world: 46
hello world: 47
hello world: 48
hello world: 49
hello world: 50
hello world: 51
hello world: 52
hello world: 53
hello world: 54
hello world: 55
hello world: 56
hello world: 57
hello world: 58
hello world: 59
hello world: 60
hello world: 61
hello world: 62
he

### 1.1.7 with 方法

事实上，**Python**提供了更安全的方法，当 `with` 块的内容结束后，**Python**会自动调用它的`close` 方法，确保读写的安全：

In [33]:
with open('newfile.txt','w') as f:
    for i in range(3000):
        x = 1.0 / (i - 1000)
        f.write('hello world: ' + str(i) + '\n')

ZeroDivisionError: float division by zero

与 `try/exception/finally` 效果相同，但更简单。

In [34]:
g = open('newfile.txt', 'r')
print (g.read())
g.close()

hello world: 0
hello world: 1
hello world: 2
hello world: 3
hello world: 4
hello world: 5
hello world: 6
hello world: 7
hello world: 8
hello world: 9
hello world: 10
hello world: 11
hello world: 12
hello world: 13
hello world: 14
hello world: 15
hello world: 16
hello world: 17
hello world: 18
hello world: 19
hello world: 20
hello world: 21
hello world: 22
hello world: 23
hello world: 24
hello world: 25
hello world: 26
hello world: 27
hello world: 28
hello world: 29
hello world: 30
hello world: 31
hello world: 32
hello world: 33
hello world: 34
hello world: 35
hello world: 36
hello world: 37
hello world: 38
hello world: 39
hello world: 40
hello world: 41
hello world: 42
hello world: 43
hello world: 44
hello world: 45
hello world: 46
hello world: 47
hello world: 48
hello world: 49
hello world: 50
hello world: 51
hello world: 52
hello world: 53
hello world: 54
hello world: 55
hello world: 56
hello world: 57
hello world: 58
hello world: 59
hello world: 60
hello world: 61
hello world: 62
he

所以，写文件时候要确保文件被正确关闭。

In [ ]:
import os
os.remove('newfile.txt')

## 1.2 StringIO和BytesIO

很多时候，数据读写不一定是文件，也可以在内存中读写。

### 1.2.1 StringIO

顾名思义就是在内存中读写str。

要把str写入StringIO，我们需要先创建一个StringIO，然后，像文件一样写入即可：

In [ ]:
from io import StringIO

f = StringIO()
f.write('hello')

In [ ]:
f.write(' ')

In [ ]:
f.write('world!')

In [ ]:
print(f.getvalue())  # getvalue()方法用于获得写入后的str。


要读取StringIO，可以用一个str初始化StringIO，然后，像读文件一样读取：

In [ ]:
from io import StringIO

f = StringIO('Hello!\nHi!\nGoodbye!')
while True:
    s = f.readline()
    if s == '':
        break
    print(s.strip())

### 1.2.2 BytesIO

StringIO操作的只能是str，如果要操作二进制数据，就需要使用BytesIO。

BytesIO实现了在内存中读写bytes，我们创建一个BytesIO，然后写入一些bytes：

In [ ]:
from io import BytesIO

f = BytesIO()

f.write('中文'.encode('utf-8'))

In [ ]:
print(f.getvalue())

请注意，写入的不是str，而是经过UTF-8编码的bytes。

和StringIO类似，可以用一个bytes初始化BytesIO，然后，像读文件一样读取：

In [ ]:
from io import BytesIO
f = BytesIO(b'\xe4\xb8\xad\xe6\x96\x87')
print(f.read())
print(f.getvalue().decode('utf-8'))

## 1.3 操作文件和目录

Python内置的os模块也可以直接调用操作系统提供的接口函数

### 1.3.1 操作系统类型

In [ ]:
import os
os.name # 操作系统类型

如果是posix，说明系统是Linux、Unix或Mac OS X，如果是nt，就是Windows系统。

要获取详细的系统信息，可以调用uname()函数：

注意uname()函数在Windows上不提供，也就是说，os模块的某些函数是跟操作系统相关的。

In [ ]:
# os.uname() 

### 1.3.2 环境变量

在操作系统中定义的环境变量，全部保存在os.environ这个变量中，可以直接查看：

In [ ]:
os.environ

要获取某个环境变量的值，可以调用os.environ.get('key')：

In [ ]:
os.environ.get('PATH')

In [ ]:
os.environ.get('x', 'default')

### 1.3.3 文件和目录

操作文件和目录的函数一部分放在os模块中，一部分放在os.path模块中，这一点要注意一下。查看、创建和删除目录可以这么调用：

In [ ]:
# 查看当前目录的绝对路径:

os.path.abspath('.')

In [ ]:
# 在某个目录下创建一个新目录，首先把新目录的完整路径表示出来:
os.path.join('F:/', 'testdir')

In [ ]:
# 然后创建一个目录:
os.mkdir('F:/testdir')

In [ ]:
# 删掉一个目录:

os.rmdir('F:/testdir')

把两个路径合成一个时，不要直接拼字符串，而要通过os.path.join()函数，这样可以正确处理不同操作系统的路径分隔符。在Linux/Unix/Mac下，os.path.join()返回这样的字符串：

part-1/part-2
而Windows下会返回这样的字符串：

part-1\part-2
同样的道理，要拆分路径时，也不要直接去拆字符串，而要通过os.path.split()函数，这样可以把一个路径拆分为两部分，后一部分总是最后级别的目录或文件名：


In [16]:
os.path.split('/Users/michael/testdir/file.txt')

('/Users/michael/testdir', 'file.txt')

os.path.splitext()可以直接让你得到文件扩展名，很多时候非常方便：

In [ ]:
os.path.splitext('/path/to/file.txt')

这些合并、拆分路径的函数并不要求目录和文件要真实存在，它们只对字符串进行操作。

文件操作使用下面的函数。假定当前目录下有一个test.txt文件：

In [ ]:
# 对文件重命名:
os.rename('test.txt', 'test.py')
# 删掉文件:
os.remove('test.py')

但是复制文件的函数居然在os模块中不存在！原因是复制文件并非由操作系统提供的系统调用。理论上讲，我们通过上一节的读写文件可以完成文件复制，只不过要多写很多代码。

幸运的是shutil模块提供了copyfile()的函数，你还可以在shutil模块中找到很多实用函数，它们可以看做是os模块的补充。

最后看看如何利用Python的特性来过滤文件。比如我们要列出当前目录下的所有目录，只需要一行代码：

最后看看如何利用Python的特性来过滤文件。比如我们要列出当前目录下的所有目录，只需要一行代码：

In [14]:
[x for x in os.listdir('.') if os.path.isdir(x)]

['.ipynb_checkpoints', 'test_dir']

要列出所有的.py文件，也只需一行代码：

In [15]:
[x for x in os.listdir('.') if os.path.isfile(x) and os.path.splitext(x)[1]=='.py']

[]

`os` 模块提供了对系统文件进行操作的方法：

In [1]:
import os

## 文件路径操作

- `os.remove(path)` 或 `os.unlink(path)` ：删除指定路径的文件。路径可以是全名，也可以是当前工作目录下的路径。
- `os.removedirs`：删除文件，并删除中间路径中的空文件夹
- `os.chdir(path)`：将当前工作目录改变为指定的路径
- `os.getcwd()`：返回当前的工作目录
- `os.curdir`：表示当前目录的符号
- `os.rename(old, new)`：重命名文件
- `os.renames(old, new)`：重命名文件，如果中间路径的文件夹不存在，则创建文件夹
- `os.listdir(path)`：返回给定目录下的所有文件夹和文件名，不包括 `'.'` 和 `'..'` 以及子文件夹下的目录。（`'.'` 和 `'..'` 分别指当前目录和父目录）
- `os.mkdir(name)`：产生新文件夹
- `os.makedirs(name)`：产生新文件夹，如果中间路径的文件夹不存在，则创建文件夹

当前目录：

In [2]:
os.getcwd()

'/home/lijin/notes-python/05. advanced python'

当前目录的符号：

In [3]:
os.curdir

'.'

当前目录下的文件：

In [4]:
os.listdir(os.curdir)

['05.01 overview of the sys module.ipynb',
 '05.05 datetime.ipynb',
 '05.13 decorator usage.ipynb',
 '.ipynb_checkpoints',
 '05.03 comma separated values.ipynb',
 '05.02 interacting with the OS - os.ipynb',
 '05.10 generators.ipynb',
 '05.15 scope.ipynb',
 '05.12 decorators.ipynb',
 '05.09 iterators.ipynb',
 'my_database.sqlite',
 '05.11 context managers and the with statement.ipynb',
 '05.16 dynamic code execution.ipynb',
 '05.14 the operator functools itertools toolz fn funcy module.ipynb',
 '05.04 regular expression.ipynb',
 '05.07 object-relational mappers.ipynb',
 '05.08 functions.ipynb',
 '05.06 sql databases.ipynb']

产生文件：

In [5]:
f = open("test.file", "w")
f.close()

print "test.file" in os.listdir(os.curdir)

True


重命名文件：

In [6]:
os.rename("test.file", "test.new.file")

print "test.file" in os.listdir(os.curdir)
print "test.new.file" in os.listdir(os.curdir)

False
True


删除文件：

In [7]:
os.remove("test.new.file")

## 系统常量

当前操作系统的换行符：

In [8]:
# windows 为 \r\n
os.linesep

'\n'

当前操作系统的路径分隔符：

In [9]:
os.sep

'/'

当前操作系统的环境变量中的分隔符（`';'` 或 `':'`）：

In [10]:
os.pathsep

':'

## 其他

`os.environ` 是一个存储所有环境变量的值的字典，可以修改。

In [11]:
os.environ["USER"]

'lijin'

`os.urandom(len)` 返回指定长度的随机字节。

## os.path 模块

不同的操作系统使用不同的路径规范，这样当我们在不同的操作系统下进行操作时，可能会带来一定的麻烦，而 `os.path` 模块则帮我们解决了这个问题。

In [12]:
import os.path

### 测试

- `os.path.isfile(path)` ：检测一个路径是否为普通文件
- `os.path.isdir(path)`：检测一个路径是否为文件夹
- `os.path.exists(path)`：检测路径是否存在
- `os.path.isabs(path)`：检测路径是否为绝对路径

### split 和 join

- `os.path.split(path)`：拆分一个路径为 `(head, tail)` 两部分
- `os.path.join(a, *p)`：使用系统的路径分隔符，将各个部分合成一个路径

### 其他

- `os.path.abspath()`：返回路径的绝对路径
- `os.path.dirname(path)`：返回路径中的文件夹部分
- `os.path.basename(path)`：返回路径中的文件部分
- `os.path.splitext(path)`：将路径与扩展名分开
- `os.path.expanduser(path)`：展开 `'~'` 和 `'~user'`

## 1.4 与操作系统进行交互：os 模块